# Initial Tear Implementation

This is *part* of the initial code that was created to implement a tear on a skinned model using Conformal Geometric Algebra, as presented in  [1] and [2]. This work set the foundation for the creation of the latest tear technique that is presented in [3].

[1] : Kamarianakis, M., & Papagiannakis, G. (2020). Deform, Cut and Tear a Skinned Model Using Conformal Geometric Algebra. In N. Magnenat-Thalmann, C. Stephanidis, E. Wu, D. Thalmann, B. Sheng, J. Kim, G. Papagiannakis, & M. Gavrilova (Eds.), Advances in Computer Graphics (Vol. 12221, pp. 434–446). Springer International Publishing. https://doi.org/10.1007/978-3-030-61864-3_37

[2] : Kamarianakis, M., & Papagiannakis, G. (2021). An All-in-One Geometric Algorithm for Cutting, Tearing, and Drilling Deformable Models. Advances in Applied Clifford Algebras, 31(3), 58. https://doi.org/10.1007/s00006-021-01151-6

[3] : Kamarianakis, M., Protopsaltis, A., Angelis, D., Tamiolakis, M., & Papagiannakis, G. (2022). Progressive tearing and cutting of soft-bodies in high-performance virtual reality. In H. Uchiyama & J.-M. Normand (Eds.), ICAT-EGVE 2022—International conference on artificial reality and telexistence and eurographics symposium on virtual environments. The Eurographics Association. https://doi.org/10.2312/egve.20221275



## Basic Code of Tear

In [ ]:
from Elements.features.SkinnedMesh.gate_module import *
import time
from collections import Counter


def euc_to_cga(v):
    """Converts a 3D Euclidean vector to conformal representation."""
    return v[0]*e1+v[1]*e2+v[2]*e3

def cga_to_euc(down):
    """Converts a 3D conformal vector to Euclidean representation."""
    return np.array([down[e1], down[e2], down[e3]])

def compare(s, t):
    """Return True if two strings have the same characters in the same amount."""
    return Counter(s) == Counter(t)

class intersection_point:
    """
    This class is used to represent an intersection point between two edges.
    Attributes:
        type: "None", "OnEdge", "OnVertex", "OnFace"
        position: the position of the intersection point
        faces: the faces that the intersection point is on
        id: the id of the vertex that the intersection point is on
        between_vertices: the two vertices that the intersection point is between
    """
    def __init__(self, type="None", position = np.array([0,0,0]), faces = [],id = -1):
        """
        Initialize the intersection point.
        """
        self.type = type
        self.position = position
        f = faces.copy()
        f.sort()
        self.faces = f
        self.id = id # to be used if the type is OnEdge
        if type == "None":
            self.exists = False
        else:
            self.exists = True
        self.between_vertices = [] # to be used if the type is OnEdge


    def __eq__(self, other):
        """
        Check if two intersection points are the same.
        """
        assert( type(other) == intersection_point)
        if self.id != -1 and other.id != -1 and self.id == other.id:
            return True
        else:
#             return (self.type == other.type) and np.allclose(self.position, other.position) and compare(self.faces, other.faces)
            return np.allclose(self.position, other.position)
    
    def __lt__(self, other):
        """
        Check if one intersection point is "smaller" than the other, in terms of the position.
        """
        assert(type(other)==intersection_point)
        if self.position[0] < other.position[0]:
            return True
        elif self.position[0] > other.position[0]:
            return False
        else: 
            if self.position[1] < other.position[1]:
                return True
            elif self.position[1] > other.position[1]:
                return False
            else:
                if self.position[2] < other.position[2]:
                    return True
                else:
                    return False  
                
    def __str__(self):
        """ 
        Print the intersection point.
        """
        if self.id != -1:
            return "Type: \"%s\", position: %s, faces: %s, vertex_id: %s" % (self.type, self.position, self.faces,self.id)
        elif self.between_vertices != [] :
            return "Type: \"%s\", position: %s, faces: %s,  between_vertices: %s" % (self.type, self.position, self.faces, self.between_vertices)
        else:
            return "Type: \"%s\", position: %s, faces: %s" % (self.type, self.position, self.faces) 



def sort2(a):
    """
    Sort a list of two elements.
    """
    return [a[0],a[1]] if a[0]<=a[1] else [a[1],a[0]]


def distance(v1,v2):
    """
    Calculate the distance between two points.
    """
    return (v1[0]-v2[0])**2 + (v1[1]-v2[1])**2 + (v1[2]-v2[2])**2

## Model optimization

In [ ]:
def same_id_with_j(j,v):
    "returns the ids i of v such that v[i]=v[j]"
    v2 , indices1 = np.unique(v,axis=0,return_inverse=True);
    return [i for i in range(len(indices1)) if indices1[i] == indices1[j]]

def min_same_id_with_j(j):
    "returns the minimum id i of v such that v[i]=v[j]."
#     v2 , indices1 = np.unique(v,axis=0,return_inverse=True);
    for i in range(len(indices1)):
        if indices1[i] == indices1[j]:
            return i
    print("error for j =", j)
    return j

def optimize_model(v,f):
    """
    Takes a vertex and face list and optimizes f: if some vertices are duplicates, 
    the faces are updated to only contain vertex ids for the min possible ids 
    """
    
    f2 = f.copy()
    v2 , indices1 = np.unique(v,axis=0,return_inverse=True);
    
    def min_same_id_with_j(j):
        "returns the minimum id i of v such that v[i]=v[j]"
        for i in range(len(indices1)):
            if indices1[i] == indices1[j]:
                return i
        print("error for j =", j)
        return j


    for i in range(len(f2)):
        for j in range(3):
            f2[i,j] = min_same_id_with_j(f2[i,j])

    return f2

## Face neighbours implementation

In [ ]:
def face_neighbours(f):
    """
    Returns a dictionary with keys = face ids and values the respective face neighbours:
    v_neighs[i] = the face ids that share only one vertex with i face
    e_neighs[i] = the face ids that share an edge with i face
    """
    v_neighs = {}
    e_neighs = {}
    for i in range(len(f)):
        v_neighs[i]=[]
        e_neighs[i]=[]
    for i in range(len(f)):
        face = f[i]
        for j in range(len(f)):
            face2 = f[j]
            if (j not in v_neighs[i]) and (j not in e_neighs[i]) : 
                if len(set(face) & set(face2))==1: 
                    v_neighs[i].append(j)
                    if i not in v_neighs[j]:
                        v_neighs[j].append(i)
                elif len(set(face) & set(face2))==2: 
                    e_neighs[i].append(j)
                    if i not in e_neighs[j]:
                        e_neighs[j].append(i)


    return v_neighs,e_neighs



In [ ]:
def point_vs_plane(point,plane):
    "Returns the side of the up-projected point with respect to the plane: +,- or 0 = on the plane"
    return (plane|point)[0]

def segment_intersection(p1, p2, q1,q2):
    """
    Returns whether the segments p (defined by p1 and p2) and q (defined by q1 and q2) intersect or not.
    If they intersect, the intersection point is also returned
    Input: p1,p2,q1,q2 must be in np.array form
    """
    
    
    A = np.array([p2-p1,q1-q2]).T
    b = np.array([q1-p1]).T
    x = np.linalg.lstsq(A,b)[0]
    if np.linalg.norm(np.dot(A,x)-b,np.inf)<1.0e-8 and 0<=x[0] and x[0]<=1 and 0<=x[1] and x[1]<=1:
        return 1, p1 +x[0]*(p2-p1)
    else:
        return 0 , 0

def x_on_segment(x, s1,s2):
    "Returns whether the euclidean point x lies on the segment bounded by s1 and s2."    
    if np.allclose(x,s1) or np.allclose(x,s2):
        return True # , 0
    n = np.cross(x-s1,x-s2)
    if not np.allclose(n,np.array([0,0,0])) :
        return False #, n
    else: # n = [0,0,0] , i.e., x,s1 and s2 are collinear, or one of them is [0,0,0] :
        if np.dot(x-s1,x-s2) < 0:
            return True#, 0
        else:
            return False#, 0

def x_on_face(x, s1, s2, s3):
    "Returns depending on the position of the  euclidean point x with respect to  "
    "the face F defined by the no-collinear points s1,s2 and s3:"
    "1 -- > x lies strictly inside F"
    "0 -- > x lies on the boundary of F"
    "-1 --> x lies outside F"
    if x_on_segment(x,s1,s2) or x_on_segment(x,s1,s2) or x_on_segment(x,s1,s2):
        return 0
    
    A = np.array([s2-s1,s3-s1]).T
    b = np.array([x-s1]).T
    res = np.linalg.lstsq(A,b)[0]
    if np.linalg.norm(np.dot(A,res)-b,np.inf)<1.0e-8 and res[0]>=0 and res[1]>=0 and res[0]+res[1]<=1:
        return 0 if ( res[0]==0 or res[1]==0 or res[0]+res[1]==1 ) else 1
    return -1
        

def orient3d(a,b,c,d):
    "returns the orientation of the tetrahedron defined by a,b,c,d"
    return np.sign(np.linalg.det(np.array([[a[0],a[1],a[2],1],[b[0],b[1],b[2],1],[c[0],c[1],c[2],1],[d[0],d[1],d[2],1]])))



def p_p_plane_intersection(p1,p2,plane):
    "Given points cga points p1,p2 that lie on different sides of a plane, returns the "
    "intersection point (in cga form) of the segment connecting p1 and p2 with the plane. "
    line = up(p1)^up(p2)^einf
    return down(intersect_line_and_plane_to_point(line.normal(),plane.dual()))






def segment_vs_face(v1,v2,face,vertices,faces):
    "Returns the intersection point of the segment defined by v1,v2 and the face:\
     - type = None : no intersection, (or tangency ?) \
     - type = Inside: the intersection point lies inside the face \
     - type = OnEdge : the segment intersects an edge of the face \
     - type = Vertex: the segment intersects an edge of the face"
    "If type is not None, the point of intersection is also returned and the list of all faces \
     that contribute to the intersection point"
    "Note that vertices and faces is the lists provided by the model"
    v1_up = up(euc_to_cga(v1))
    v2_up = up(euc_to_cga(v2))
    f = faces[face]
    fv0 = vertices[f[0]]
    fv1 = vertices[f[1]]
    fv2 = vertices[f[2]]
    f0_up = up(euc_to_cga(fv0))
    f1_up = up(euc_to_cga(fv1))
    f2_up = up(euc_to_cga(fv2))
    f_plane = (f0_up^f1_up^f2_up^einf).dual()
#     f_plane = f0_up^f1_up^f2_up^einf
    s1 = np.sign(point_vs_plane(v1_up,f_plane))
    s2 = np.sign(point_vs_plane(v2_up,f_plane))
    
    
    if s2 <0 and s1 > 0: #MAY HAVE TO SWAP IF THE ORIENTATION OF THE FACES IS REVERSED

        # check for intersection with vertices of face
        if x_on_segment(fv0,v1,v2):
            vertex_neighbours = [f2 for f2 in v_n[face] if f[0] in faces[f2]]
            vertex_neighbours.append(face)
            p = intersection_point("Vertex", fv0, vertex_neighbours) # fv0 is the intersection point
            p.id = f[0]
            return p           
        if x_on_segment(fv1,v1,v2):
            vertex_neighbours = [f2 for f2 in v_n[face] if f[1] in faces[f2]]
            vertex_neighbours.append(face)
            p = intersection_point("Vertex", fv1, vertex_neighbours) # fv1 is the intersection point
            p.id = f[1]
            return p
        if x_on_segment(fv2,v1,v2):
            vertex_neighbours = [f2 for f2 in v_n[face] if f[2] in faces[f2]]
            vertex_neighbours.append(face)
            p = intersection_point("Vertex", fv2, vertex_neighbours) # fv2 is the intersection point
            p.id = f[2]
            return p

        x = cga_to_euc(p_p_plane_intersection(euc_to_cga(v1),euc_to_cga(v2),f_plane)) # the intersection of the plane with the line

#         print(x,fv0,fv1,fv2)
        
        if x_on_segment(x,fv0,fv1):
            edge_neighbours = [f2 for f2 in e_n[face] if f[0] in faces[f2] and f[1] in faces[f2]]
            edge_neighbours.append(face)
            if len(edge_neighbours) > 2:
                print("face : ", face)
                print("e_n[face] : ", e_n[face])
                print("edge_neighbours : ", edge_neighbours)
                assert(len(edge_neighbours)==2)
            p = intersection_point("OnEdge", np.array(x),edge_neighbours) # we are on the boundary
            p.between_vertices = sort2([f[0],f[1]]) 
            return p
        if x_on_segment(x,fv1,fv2):
            edge_neighbours = [f2 for f2 in e_n[face] if f[1] in faces[f2] and f[2] in faces[f2]]
            edge_neighbours.append(face)
            if len(edge_neighbours) > 2:
                print("face : ", face)
                print("e_n[face] : ", e_n[face])
                print("edge_neighbours : ", edge_neighbours)
                assert(len(edge_neighbours)==2)
            p = intersection_point("OnEdge", np.array(x),edge_neighbours) # we are on the boundary
            p.between_vertices = sort2([f[2],f[1]]) 
            return p
        if x_on_segment(x,fv2,fv0):
            edge_neighbours = [f2 for f2 in e_n[face] if f[2] in faces[f2] and f[0] in faces[f2]]
            edge_neighbours.append(face)
            if len(edge_neighbours) > 2:
                print("face : ", face)
                print("e_n[face] : ", e_n[face])
                print("edge_neighbours : ", edge_neighbours)
                assert(len(edge_neighbours)==2)
            p = intersection_point("OnEdge", np.array(x),edge_neighbours) # we are on the boundary
            p.between_vertices = sort2([f[0],f[2]]) 
            return p


        # below, we assume no boundary intersection and check 
        orient = orient3d(fv0,fv1,fv2,v1)
        ind01 =  orient3d(fv0,x,fv1,v1)
#         print (ind01, orient3d(x,fv1,fv2,v1),orient3d(fv0,x,fv2,v1)) #-,+,+
        if ind01 == -orient and ind01 == -orient3d(x,fv1,fv2,v1) and ind01 == -orient3d(fv2,fv0,x,v1):
#             print(face, orient,ind01, orient3d(x,fv1,fv2,v1),orient3d(fv2,fv0,x,v1))
            return intersection_point("Inside", np.array(x),[face]) # we are inside face
        else:
#             print("test A")
            return intersection_point("None") # we are outside face
    else: 
#         print("test")
        return intersection_point("None")


def draw():
    "for block below"
    p = mp.plot(newv, f,newv[:, 1],shading={"scale": 2.5,"wireframe":False},return_plot=True)
    p.add_points(np.array([v1]), shading={"point_size": 0.4,"point_color": "red"} );
    p.add_points(np.array([v2]), shading={"point_size": 0.4,"point_color": "blue"} );
    p.add_lines(np.array([v1]), np.array([v2]), shading={"line_color": "black"});
    # print(np.array([f[0]]))

    if len(f_pierced):
        p.add_points(np.array(possible_intersections),shading={"point_size": 0.4,"point_color": "yellow"})
        p.add_lines(newv[f_pierced[:,0]], newv[f_pierced[:,1]], shading={"line_color": "magenta"});
        p.add_lines(newv[f_pierced[:,2]], newv[f_pierced[:,1]], shading={"line_color": "magenta"});
        p.add_lines(newv[f_pierced[:,2]], newv[f_pierced[:,0]], shading={"line_color": "magenta"});

In [ ]:
######### obsolete - start #############
class face_intersection:
    """
    This class is used to represent the intersection between a face and a plane.
    Attributes:
        type: "None", "Tangency", "Normal", "SinglePoint", "WholeEdge", "ThroughVertex"
        inter_points: a list of intersection points
        positive_vertices: the vertices of the face that lie on the positive side of the plane
        negative_vertices: the vertices of the face that lie on the negative side of the plane
        on_plane_vertices: the vertices of the face that lie on the plane
    Details on type:
        None: there is no intersection
        Tangency: plane is tangent with face
        Normal: there are 2 internal intersection points on two different edges of the face
        SinglePoint: a single vertex
        WholeEdge: an entire edge is on the plane
        ThroughVertex: a vertex and a point OnEdge
    """


    def __init__(self, type="None", inter_points = []):
        self.type = type
        self.inter_points = inter_points
        self.positive_vertices = []
        self.negative_vertices = []
        self.on_plane_vertices = []
    
    def __str__(self):
        return "Type: %s, positive_vertices: %s, negative_vertices: %s, on_plane_vertices: %s " \
                %(self.type,self.positive_vertices,self.negative_vertices,self.on_plane_vertices )
    

######### obsolete - end #############


def face_vs_plane(face_id,plane,v,f):
    "given a mesh (v,f), a plane and a face_id of f, we determine the intersection point(s) "
    "of face_id with the plane. "
    inter_points = []
    res = face_intersection()

    fv = [v[f[face_id][i]] for i in range(3)]
    fv_vs_plane = [np.sign(point_vs_plane(up(euc_to_cga(fv[i])),plane)) for i in range(3)]


    for i in range(3):
        if fv_vs_plane[i] == 0:
            res.on_plane_vertices.append(i)
        if fv_vs_plane[i] == 1:
            res.positive_vertices.append(i)
        if fv_vs_plane[i] == -1:
            res.negative_vertices.append(i)

    def neigh_face(i): # used when an intersection point is a Vertex
        return [f2 for f2 in v_n[face_id] + e_n[face_id] if f[face_id][i] in f[f2]]+[face_id]
    
    if len(res.on_plane_vertices) == 3:
        res.type = "Tangency"
        res.inter_points = [intersection_point("tangency",np.array(fv[i])) for i in res.on_plane_vertices]
        return res
    
    elif len(res.on_plane_vertices) == 2:
        res.type = "WholeEdge"  
        res.inter_points = [intersection_point("Vertex",np.array(fv[i]),neigh_face(i),f[face_id][i]) for i in res.on_plane_vertices]
        return res

    elif len(res.on_plane_vertices) == 1:
        res.inter_points = [intersection_point("Vertex",np.array(fv[i]),neigh_face(i),f[face_id][i]) for i in res.on_plane_vertices]
        if len(res.positive_vertices) == 1: 
            assert(len(res.negative_vertices) == 1)
            pos = res.positive_vertices[0]
            neg = res.negative_vertices[0]
            v_pos = euc_to_cga(fv[pos])
            v_neg = euc_to_cga(fv[neg])
            intersection_pos = np.array(cga_to_euc(p_p_plane_intersection(v_pos,v_neg,plane)))
            neigh_face  = [f2 for f2 in e_n[face_id] if f[face_id][pos] in f[f2] and f[face_id][neg] in f[f2]]
            neigh_face.append(face_id)
            p = intersection_point("OnEdge",intersection_pos,neigh_face)
            p.between_vertices = sort2([ f[face_id][res.positive_vertices[0]], f[face_id][res.negative_vertices[0]] ]) 
            res.inter_points.append(p)   
            res.type = "ThroughVertex"
            return res
        else:
            assert (len(res.negative_vertices) == 2 or len(res.positive_vertices) == 2)
            res.type = "SinglePoint"
            return res
        
    elif len(res.positive_vertices) == 3 or len(res.negative_vertices) == 3:
        res.type = "None"
        return res
    
    elif len(res.positive_vertices) == 1 and len(res.negative_vertices) == 2:
        pos = res.positive_vertices[0]
        neg1 = res.negative_vertices[0]
        neg2 = res.negative_vertices[1]
        v_pos = euc_to_cga(fv[pos])
        v_neg1 = euc_to_cga(fv[neg1])
        v_neg2 = euc_to_cga(fv[neg2])
        intersection_pos = np.array(cga_to_euc(p_p_plane_intersection(v_pos,v_neg1,plane)))
        neigh_face  = [f2 for f2 in e_n[face_id] if f[face_id][pos] in f[f2] and f[face_id][neg1] in f[f2]]
        neigh_face.append(face_id)
        p = intersection_point("OnEdge",intersection_pos,neigh_face) 
        p.between_vertices = sort2([ f[face_id][pos], f[face_id][neg1] ])
        res.inter_points = [p] 
        intersection_pos = np.array(cga_to_euc(p_p_plane_intersection(v_pos,v_neg2,plane)))
        neigh_face  = [f2 for f2 in e_n[face_id] if f[face_id][pos] in f[f2] and f[face_id][neg2] in f[f2]]
        neigh_face.append(face_id)
        q = intersection_point("OnEdge",intersection_pos,neigh_face)
        q.between_vertices = sort2([ f[face_id][pos], f[face_id][neg2] ]) 
        res.inter_points.append(q)  
        res.type = "Normal"
        return res
    elif len(res.positive_vertices) == 2 and len(res.negative_vertices) == 1:
        pos1 = res.positive_vertices[0]
        pos2 = res.positive_vertices[1]
        neg = res.negative_vertices[0]
        v_pos1 = euc_to_cga(fv[pos1])
        v_pos2 = euc_to_cga(fv[pos2])
        v_neg = euc_to_cga(fv[neg])
        intersection_pos = np.array(cga_to_euc(p_p_plane_intersection(v_pos1,v_neg,plane)))
        neigh_face  = [f2 for f2 in e_n[face_id] if f[face_id][pos1] in f[f2] and f[face_id][neg] in f[f2]]
        neigh_face.append(face_id)
        p = intersection_point("OnEdge",intersection_pos,neigh_face) 
        p.between_vertices = sort2([ f[face_id][pos1], f[face_id][neg] ])        
        res.inter_points = [p] 

        intersection_pos = np.array(cga_to_euc(p_p_plane_intersection(v_pos2,v_neg,plane)))
        neigh_face  = [f2 for f2 in e_n[face_id] if f[face_id][pos2] in f[f2] and f[face_id][neg] in f[f2]]
        neigh_face.append(face_id)
        q = intersection_point("OnEdge",intersection_pos,neigh_face)
        res.inter_points.append(q)  
        q.between_vertices = sort2([ f[face_id][pos2], f[face_id][neg] ])  
        res.type = "Normal"
        return res
    else:
        assert (False)

In [ ]:

def inter_distance(X,Y):
    "given intersection points X and Y, calculate their squared euclidean distance"
    return distance(X.position,Y.position)


def connect_inter_points(X,Y,plane,v,f):
    "given the intersection points X,Y of a mesh (v,f) return the a list of intersection points "
    "of the mesh with the plane that lie in between X and Y."
    "Prerequisite: X and Y points' type are either Vertex or OnEdge"
    if X == Y: 
        return [X,Y]

faces_checked = []
faces_checked_result = []    
    
def initialize_face_res():    
    global faces_checked
    global faces_checked_result
    faces_checked = []
    faces_checked_result = []


def save_res(face,res):
    global faces_checked
    global faces_checked_result
    "saves the face_conflict result of face (id) against the checked plane"
    if face not in faces_checked:
        faces_checked.append(face)
        faces_checked_result.append(res)

def find_next_dot(X,plane,v,f,closest_to = intersection_point("None")):
    "Given a intersection point X that is either Vertex or OnEdge, determine the two adjacent inter points"
    "If a 4th arguement is provided that is a valid intersection point it only returns the "
    "adjacent inter point closest to the argument"
    
    Y = closest_to
    
    
    if "None" in [X.type,Y.type]:
        print("None given as input")
        return intersection_point()
    
    intersections = []
    count = 0
    for face in X.faces:
        res = face_vs_plane(face,plane,v,f)
        save_res(face,res)
        for p in res.inter_points:
            if p != X and p not in intersections:
                intersections.append(p)
                count = count +1
                if count == 3:
                    break
#         intersections = intersections + res.inter_points
#     return [p for p in np.unique(intersections) if p != X] 
    if closest_to.type == "None":
        return intersections
    else:
        if len(intersections) in [0] :
#             print("got here")
#             return intersections
            print("find_next_dot did not find a point for \nX: ", X, "and Y: ", Y)
            return intersection_point()
        elif len(intersections) in [1] :
            return intersections[0]
        else:
            assert(len(intersections) in [2])
            return intersections[0] if inter_distance(intersections[0],closest_to) <= inter_distance(intersections[1],closest_to) else intersections[1]







In [ ]:
def complete_stich(X,Y,plane,v,f):
    "Given the intersection points X,Y of the mesh (v,f) that lie on the plane"
    "it returns a complete list of intersection points from X to Y"
    
    
    if "None" in [X.type,Y.type]:
        return []
    if X == Y:
        return [X]
    
    if X.type in ["Inside","OnEdge"] and Y.type == X.type:
        if compare(X.faces,Y.faces): 
#             M = intersection_point(X.type,mid_point(X.position,Y.position),X.faces)
#             return [X,M,Y] # added a middle point for tearing purposes
            return [X,Y] 
    if X.type == "Inside":
        Z = find_next_dot(X,plane,v,f,Y)
        if Z == Y :
#             M = intersection_point("Inside",mid_point(X.position,Y.position),X.faces)
#             return [X,M,Y] # added a middle point for tearing purposes
            return [X,Y] 
        else:
#             print("@",X)
#             print("@",Z)
            return [X] + complete_stich(Z,Y,plane,v,f)
    
    if Y.type == "Inside":
        Z = find_next_dot(Y,plane,v,f,X)
        if Z == X :
#             M = intersection_point("Inside",mid_point(X.position,Y.position),Y.faces)
#             return [X,M,Y] # added a middle point for tearing purposes
            return [X,Y]
        else:
            return complete_stich(X,Z,plane,v,f) + [Y]
        
    if Y.type == "OnEdge":
        Z = find_next_dot(Y,plane,v,f,X)
        if Z == X :
#             M = intersection_point("Inside",mid_point(X.position,Y.position),Y.faces)
#             return [X,M,Y] # added a middle point for tearing purposes
            return [X,Y] 
        elif Z.type == "Vertex":
            return complete_stich(X,Z,plane,v,f) + [Y]
        else:
            ;
        
    assert (X.type not in ["Inside"])
    assert (Y.type not in ["Inside"])
    
    next_dot = X
    L = [X]
    count = 0
    while next_dot != Y and count <=30:
        next_dot = find_next_dot(next_dot,plane,newv,f,Y)
        L.append(next_dot)
        count = count + 1
    return L


def segment_vs_mesh(v1,v2,v,f):
    "detects the intersection of segment defined by v1 and v2 with the mesh v,f"
    "return the face_id of the face intersected and the intersection point"
    for i in range(len(f)):
        X = segment_vs_face(v1,v2,i,v,f)
        if X.type != "None":
            return X
    return intersection_point()

In [ ]:
def populate_duo_list(L):
    "Given a list of only two intersection points, inserts one middle point for tearing purposes"
    "Otherwise the list itself is returned"
    if len(L) != 2:
        return L
    
    def mid_point(a,b):
        "given two arrays a,b returns their average"
        return (a+b)/2

    if L[0].type == "Inside":
        M = intersection_point("Inside",mid_point(L[0].position,L[1].position),L[0].faces)
        return [L[0],M,L[1]] 
    elif L[1].type == "Inside":
        M = intersection_point("Inside",mid_point(L[0].position,L[1].position),L[1].faces)
        return [L[0],M,L[1]] 
    elif L[0].type == L[0].type and L[0].type in ["OnEdge"]:
        M = intersection_point("OnEdge",mid_point(L[0].position,L[1].position),L[1].faces)
        return [L[0],M,L[1]] 
    elif L[0].type == L[0].type and L[0].type in ["Vertex"]:
        M = intersection_point("OnEdge",mid_point(L[0].position,L[1].position), list(set( L[0].faces) & set( L[1].faces)))
        return [L[0],M,L[1]] 


# global finalv = newv
# global finalf = f
# global P1 = s1
# global P2 = s2
# global P3 = X.position
# global finalL = L

def tear(o1,o2,s1,s2,v,f,draw = False,return_plane = False, return_normal = False): 
    point_sizee = 2.5
    "given an original segment (o1,o2) and a final (s1,s2) along with a mesh (v,f)"
    "we return the list of intersection points from initial cut point to the final cut point"
    
    start = time.time()
    X = segment_vs_mesh(o2,o1,newv,f)
    Y = segment_vs_mesh(s2,s1,newv,f)
    end = time.time()
    print("2 X segment_vs_mesh time: ", end-start)
    
    if "None" in [X.type,Y.type]:
        if draw :
            p = mp.plot(newv, f,newv[:, 1],shading={"scale": 2.5,"wireframe":True},return_plot=True)
            p.add_points(np.array([X.position]), shading={"point_size": point_sizee,"point_color": "red"});
            p.add_points(np.array([Y.position]), shading={"point_size": point_sizee,"point_color": "blue"});
        print("No tearing")
        if return_plane:
            return [], 0, np.array([0,0,0])
        else:
            return [], 0, np.array([0,0,0])
#         return [v,f]
    
    f0_up = up(euc_to_cga(X.position))
    f1_up = up(euc_to_cga(s1))
    f2_up = up(euc_to_cga(s2))
    plane = (f0_up^f1_up^f2_up^einf).dual()
    plane_normal = np.cross(s2-s1,X.position-s1)
    if not np.allclose(plane_normal,np.array([0,0,0])):
        plane_normal = plane_normal / np.linalg.norm(plane_normal)
#     print("plane_normal", plane_normal)
    
    start = time.time()
    L = complete_stich(X,Y,plane,newv,f)
    end = time.time()
    print("complete_stich time: ", end-start)
    start = time.time()
    L = populate_duo_list(L)
    end = time.time()
    print("populate_duo_list time: ", end-start)
    
    global finalv
    global finalf
    global P1
    global P2
    global P3
    global finalL
    
    finalv = newv
    finalf = f
    P1 = s1
    P2 = s2
    P3 = X.position
    finalL = L
    
    if draw  :
        p = mp.plot(newv, f,newv[:, 1],shading={"flat": False, "scale": 2.5,"wireframe":True},return_plot=True)
        p.add_lines(X.position, s1, shading={"line_color": "red"});
        p.add_lines(X.position, s2, shading={"line_color": "red"});
        p.add_lines(s1,s2, shading={"line_color": "red"});
#         global final 
        if len(L):
            p.add_points(np.array([p.position for p in L]),shading={"point_size": point_sizee,"point_color": "yellow"})
#         for p in L:
#             print("-", p)
        
    if return_plane:
        if return_normal:
            return L, plane, plane_normal
        else:
            return L, plane, np.array([0,0,0])
    else:
        return L, 0, np.array([0,0,0])
    



In [ ]:
def common_vertices(face1,face2,faces):
    "return the common vertices that appear in face1 and face2 in the mesh (v,faces)"
    return list(set(faces[face1]) & set(faces[face2]))

def common_elements(l1,l2):
    "returns a list of all common elements of lists l1 and l2 "
    return list(set(l1) & set(l2))
# print(common_vertices(0,1,f))

def x_bary(x, s1, s2, s3):
    "Returns the barycentric coordinates of x with respect to the no-collinear points s1,s2 and s3"
    A = np.array([s2-s1,s3-s1]).T
    b = np.array([x-s1]).T
    res = np.linalg.lstsq(A,b)[0]
#     print("x_bary_res = ", res)
    tol = 1.0e-8
    if np.linalg.norm(np.dot(A,res)-b,np.inf)<1.0e-8 and res[0][0]>=-tol\
      and res[1]>=-tol and res[0][0]+res[1][0]<=1+tol:
        return [1-res[0][0]-res[1][0], res[0][0], res[1][0]]
    else:
        print(x,s1,s2,s3)
        print(np.linalg.norm(np.dot(A,res)-b,np.inf), res[0],res[1])
        assert(False)

def normalize_weights(indices,weights):
    if len(indices) == 4:
        "Just to make sure that weights are ordered from larger to smaller"
        max_weight_indices = sorted(range(len(weights)), key = lambda sub: weights[sub])[-4:] # change 4=> any number
        max_weight_indices.reverse()
        new_weights = [weights[i] for i in max_weight_indices]
        return np.array([indices[i] for i in max_weight_indices]),np.array([weights[i] for i in max_weight_indices])
    elif len(indices) < 4:
        in2 = list(indices)
        ws2 = list(weights)
#         print(in2)
        for i in range(4-len(indices)):
            in2.append(-1)
            ws2.append(0)
        return np.array(in2), np.array(ws2)
    else:
        "we have more than 4 influencial bones. We pick the 4 most influencial and remove the others"
        "the 4 weights that remain are normalized st their sum equals 1"
        in2 = indices.copy()
        ws2 = weights.copy()
        max_weight_indices = sorted(range(len(ws2)), key = lambda sub: ws2[sub])[-4:] # change 4=> any number
        max_weight_indices.reverse()
        new_weights = [ws2[i] for i in max_weight_indices]
        s = sum(new_weights)
#         print("hey")
        return np.array([in2[i] for i in max_weight_indices]),np.array([ws2[i]/s for i in max_weight_indices])




def single_weight_update(L,i,v,f):
    "Returns the weight of i-th added tear-point"
    if L[i].type == "Vertex":
#         print( vw.id[L[i].id], vw.weight[L[i].id])
        return vw.id[L[i].id], vw.weight[L[i].id]
    elif L[i].type in ["Inside"]:
        inside_face = L[i].faces[0]
#         print(inside_face)
        coef =  x_bary( L[i].position, v[f[inside_face][0]], v[f[inside_face][1]],v[f[inside_face][2]] ) 
        indices = [ list(vw.id[f[inside_face][i]]) for i in range(3)]
        weights = [ vw.weight[f[inside_face][i]] for i in range(3)]
        ind_appearing = np.unique([i for s in indices for i in s])
        ws = []
        for i in range(len(ind_appearing)):
            ws.append(sum([coef[j]*weights[j][indices[j].index(ind_appearing[i])] for j in range(3) if ind_appearing[i] in indices[j]]))
        return normalize_weights(ind_appearing, np.array(ws))
    elif L[i].type in ["OnEdge"]:
        neigh_vert_0 = L[i].between_vertices[0]
        neigh_vert_1 = L[i].between_vertices[1]
        
        vA = v[neigh_vert_0]
        vB = L[i].position
        vC = v[neigh_vert_1]
#         coef =  [distance(L[i].position,v[neigh_vert_0]), distance(L[i].position,v[neigh_vert_1]) ]
#         coef = coef/sum(coef)
        
        if vA[0] != vC[0]:
            coef0 = (vB[0]-vC[0])/(vA[0]-vC[0])
        elif vA[1] != vC[1]:
            coef0 = (vB[1]-vC[1])/(vA[1]-vC[1])
        else:
            coef0 = (vB[2]-vC[2])/(vA[2]-vC[2])
        coef = [coef0,1-coef0]
        indices = [ list(vw.id[neigh_vert_0]), list(vw.id[neigh_vert_1]) ]
        weights = [ list(vw.weight[neigh_vert_0]), list(vw.weight[neigh_vert_1])]
        ind_appearing = np.unique([i for s in indices for i in s])
        ws = []
        for i in range(len(ind_appearing)):
            ws.append(sum([coef[j]*weights[j][indices[j].index(ind_appearing[i])] for j in range(2) if ind_appearing[i] in indices[j]]))
        return normalize_weights(ind_appearing, np.array(ws))







In [ ]:
def single_stich(o1,o2,s1,s2,v,f,draw = False, open_up = 0):
    "performs the tear and retriangulates correctly"
    
    
    global faces_checked
    global faces_checked_result
#     faces_checked = []
#     faces_checked_result = []


    start = time.time()
    L, plane, n = tear(o1,o2,s1,s2,v,f,draw,True,True) # True,True --> it also returns the plane its normal
    end = time.time()
    print("tear time: ", end-start)
    
    def where_is_i_vertex_of_face(i,face_id):
        "returns +1/-1/0 that denotes the position of face_id[i] vertex against the plane"
        fv = v[f[face_id][i]]
        return np.sign(point_vs_plane(up(euc_to_cga(fv)),plane))
    
    def face_res(face_id):
        return faces_checked_result[faces_checked.index(face_id)]

    if len(L) == 0 :
        return [], v, f , []
    v2 = v.copy()
    f2 = f.copy()
    
    m = len(L)
    count_v = len(v)
    count_v2 = len(v)+m-1
    redundant_faces = [] # these faces no longer exist in the teared mesh, and should not be printed
    added_faces = []
    


    for i in range(m):
        if i == 0 or i == m-1 :
            v2 = np.append(v2, np.array([L[i].position]),axis = 0 )
        else:
            v2 = np.append(v2, np.array([L[i].position]) - open_up * n,axis = 0 )
        vw.id[count_v+i], vw.weight[count_v+i] = single_weight_update(L,i,v,f)
    for i in range(1,m-1): # all torn points, except source and end, will appear twice
        v2 = np.append(v2, np.array([L[i].position]) + open_up * n,axis = 0 )
#         print("2nd loop i", i)
        vw.id[count_v2+i], vw.weight[count_v2+i] = single_weight_update(L,i,v,f)

                    
                
    if L[-1].type == "Inside": # and L[1].type == "OnEdge":
        inside_face = L[-1].faces[0]
        redundant_faces.append(inside_face)
        m = len(L)
        if L[-2].type == "OnEdge":
            if f[inside_face][0] not in L[-2].between_vertices:
                f2 = np.append(f2, np.array([[ f[inside_face][0] ,count_v+m-1 ,  f[inside_face][2] ]]),axis = 0 )
                f2 = np.append(f2, np.array([[ f[inside_face][0] , f[inside_face][1], count_v+m-1  ]]),axis = 0 )
                f2 = np.append(f2, np.array([[ count_v+m-1, count_v+m-2,  f[inside_face][2]  ]]),axis = 0 )
                f2 = np.append(f2, np.array([[ count_v+m-1, f[inside_face][1] , count_v2 +m-2 ]]),axis = 0 )
            elif f[inside_face][1] not in L[-2].between_vertices:
                f2 = np.append(f2, np.array([[ f[inside_face][1] ,count_v+m-1 ,  f[inside_face][0] ]]),axis = 0 )
                f2 = np.append(f2, np.array([[ f[inside_face][1] , f[inside_face][2], count_v+m-1  ]]),axis = 0 )
                f2 = np.append(f2, np.array([[ count_v+m-1, count_v +m-2,  f[inside_face][0]  ]]),axis = 0 )
                f2 = np.append(f2, np.array([[ count_v+m-1, f[inside_face][2] , count_v2 +m-2 ]]),axis = 0 )
            elif f[inside_face][2] not in L[-2].between_vertices:
                f2 = np.append(f2, np.array([[ f[inside_face][2] ,count_v+m-1 ,  f[inside_face][1] ]]),axis = 0 )
                f2 = np.append(f2, np.array([[ f[inside_face][2] , f[inside_face][0], count_v+m-1  ]]),axis = 0 )
                f2 = np.append(f2, np.array([[ count_v+m-1, count_v +m-2,  f[inside_face][1]  ]]),axis = 0 )
                f2 = np.append(f2, np.array([[ count_v+m-1, f[inside_face][0] , count_v2 +m-2 ]]),axis = 0 )
            else:
                assert(False)


            
    for j in range(1,len(L)-2):
        if L[j].type == "OnEdge" and L[j+1].type == "OnEdge":
            inside_face = common_elements(L[j].faces,L[j+1].faces)
            assert(len(inside_face)==1)
            inside_face = inside_face[0]
            redundant_faces.append(inside_face)
            pos = face_res(inside_face).positive_vertices
            neg = face_res(inside_face).negative_vertices
            if pos == [0]:
                if f[inside_face][1] in L[j].between_vertices: # L[j] lies on [0,1] edge of inside_face
                    X = count_v+j;   Y = count_v+j+1; 
                    Xn = X + len(L) - 1; Yn = Y + len(L) - 1
                elif f[inside_face][2] in L[j].between_vertices: # L[j] lies on [0,2] edge of inside_face
                    X = count_v+j+1; Y = count_v+j
                    Xn = X + len(L) - 1; Yn = Y + len(L) - 1
                else:
                    assert(False)
                f2 = np.append(f2, np.array([[ f[inside_face][0], X , Y ]]),axis = 0 )
                f2 = np.append(f2, np.array([[ Xn , f[inside_face][1],  f[inside_face][2] ]]),axis = 0 )
                f2 = np.append(f2, np.array([[ Xn , f[inside_face][2],  Yn ]]),axis = 0 )    
            elif pos == [1]:
                if f[inside_face][2] in L[j].between_vertices: # L[j] lies on [1,2] edge of inside_face
                    X = count_v+j; Y = count_v+j+1
                    Xn = X + len(L) - 1; Yn = Y + len(L) - 1
                elif f[inside_face][0] in L[j].between_vertices: # L[j] lies on [1,2] edge of inside_face
                    X = count_v+j+1; Y = count_v+j
                    Xn = X + len(L) - 1; Yn = Y + len(L) - 1
                else:
                    assert(False)
                f2 = np.append(f2, np.array([[ f[inside_face][1], X , Y ]]),axis = 0 )
                f2 = np.append(f2, np.array([[ Xn , f[inside_face][2],  f[inside_face][0] ]]),axis = 0 )
                f2 = np.append(f2, np.array([[ Xn , f[inside_face][0],  Yn ]]),axis = 0 )    
            elif pos == [2]:
                if f[inside_face][0] in L[j].between_vertices: # L[j] lies on [0,2] edge of inside_face
                    X = count_v+j; Y = count_v+j+1
                    Xn = X + len(L) - 1; Yn = Y + len(L) - 1
                elif f[inside_face][1] in L[j].between_vertices: # L[j] lies on [1,2] edge of inside_face
                    X = count_v+j+1; Y = count_v+j
                    Xn = X + len(L) - 1; Yn = Y + len(L) - 1
                else:
                    assert(False)
                f2 = np.append(f2, np.array([[ f[inside_face][2], X , Y ]]),axis = 0 )
                f2 = np.append(f2, np.array([[ Xn , f[inside_face][0],  f[inside_face][1] ]]),axis = 0 )
                f2 = np.append(f2, np.array([[ Xn , f[inside_face][1],  Yn ]]),axis = 0 )
            elif neg == [0]:
                if f[inside_face][1] in L[j].between_vertices: # L[j] lies on [0,1] edge of inside_face
                    X = count_v+j; Y = count_v+j+1
                    Xn = X + len(L) - 1; Yn = Y + len(L) - 1
                elif f[inside_face][2] in L[j].between_vertices: # L[j] lies on [0,2] edge of inside_face
                    X = count_v+j+1; Y = count_v+j
                    Xn = X + len(L) - 1; Yn = Y + len(L) - 1
                else:
                    assert(False)
                f2 = np.append(f2, np.array([[ f[inside_face][0], Xn , Yn ]]),axis = 0 )
                f2 = np.append(f2, np.array([[ X , f[inside_face][1],  f[inside_face][2] ]]),axis = 0 )
                f2 = np.append(f2, np.array([[ X , f[inside_face][2],  Y ]]),axis = 0 )
            elif neg == [1]:
                if f[inside_face][2] in L[j].between_vertices: # L[j] lies on [2,1] edge of inside_face
                    X = count_v+j;  Y = count_v+j+1
                    Xn = X + len(L) - 1; Yn = Y + len(L) - 1
                elif f[inside_face][0] in L[j].between_vertices: # L[j] lies on [0,1] edge of inside_face
                    X = count_v+j+1;  Y = count_v+j
                    Xn = X + len(L) - 1; Yn = Y + len(L) - 1
                else:
                    assert(False)
                f2 = np.append(f2, np.array([[ f[inside_face][1], Xn , Yn ]]),axis = 0 )
                f2 = np.append(f2, np.array([[ X , f[inside_face][2],  f[inside_face][0] ]]),axis = 0 )
                f2 = np.append(f2, np.array([[ X , f[inside_face][0],  Y ]]),axis = 0 )
            elif neg == [2]:
                if f[inside_face][0] in L[j].between_vertices: # L[j] lies on [2,0] edge of inside_face
                    X = count_v+j;  Y = count_v+j+1
                    Xn = X + len(L) - 1; Yn = Y + len(L) - 1
                elif f[inside_face][1] in L[j].between_vertices: # L[j] lies on [2,1] edge of inside_face
                    X = count_v+j+1;  Y = count_v+j
                    Xn = X + len(L) - 1; Yn = Y + len(L) - 1
                else:
                    assert(False)
                f2 = np.append(f2, np.array([[ f[inside_face][2], Xn , Yn ]]),axis = 0 )
                f2 = np.append(f2, np.array([[ X , f[inside_face][0],  f[inside_face][1] ]]),axis = 0 )
                f2 = np.append(f2, np.array([[ X , f[inside_face][1],  Y ]]),axis = 0 )


#                 f2 = np.append(f2, np.array([[ count2_v+j , count_v+j+1 , f[inside_face][neg[0]] ]]),axis = 0 )
#                 f2 = np.append(f2, np.array([[ count_v+j , count_v+j+1 , f[inside_face][neg[0]] ]]),axis = 0 )
#                 f2 = np.append(f2, np.array([[ count_v+j , count_v+j+1 , f[inside_face][neg[0]] ]]),axis = 0 )
#             elif len(pos) == 1:
#                 if pos == 0:
#                     if 
#                 f2 = np.append(f2, np.array([[ count_v+j , count_v+j+1 , f[inside_face][pos[0]] ]]),axis = 0 )
    if L[0].type == "Inside" and L[1].type == "OnEdge": 
        inside_face = L[0].faces[0]
        redundant_faces.append(inside_face)
        if f[inside_face][0] not in L[1].between_vertices:
            f2 = np.append(f2, np.array([[ f[inside_face][0] ,count_v ,  f[inside_face][2] ]]),axis = 0 )
            f2 = np.append(f2, np.array([[ f[inside_face][0] , f[inside_face][1], count_v  ]]),axis = 0 )
            f2 = np.append(f2, np.array([[ count_v, count_v2 + 1,  f[inside_face][2]  ]]),axis = 0 )
            f2 = np.append(f2, np.array([[ count_v, f[inside_face][1] , count_v + 1 ]]),axis = 0 )
        elif f[inside_face][1] not in L[1].between_vertices:
            f2 = np.append(f2, np.array([[ f[inside_face][1] ,count_v ,  f[inside_face][0] ]]),axis = 0 )
            f2 = np.append(f2, np.array([[ f[inside_face][1] , f[inside_face][2], count_v  ]]),axis = 0 )
            f2 = np.append(f2, np.array([[ count_v, count_v2 + 1,  f[inside_face][0]  ]]),axis = 0 )
            f2 = np.append(f2, np.array([[ count_v, f[inside_face][2] , count_v + 1 ]]),axis = 0 )
        elif f[inside_face][2] not in L[1].between_vertices:
            f2 = np.append(f2, np.array([[ f[inside_face][2] ,count_v ,  f[inside_face][1] ]]),axis = 0 )
            f2 = np.append(f2, np.array([[ f[inside_face][2] , f[inside_face][0], count_v  ]]),axis = 0 )
            f2 = np.append(f2, np.array([[ count_v, count_v2 + 1,  f[inside_face][1]  ]]),axis = 0 )
            f2 = np.append(f2, np.array([[ count_v, f[inside_face][0] , count_v + 1 ]]),axis = 0 )
        else:
            assert(False)
    
    if L[0].type == "Inside" and L[1].type == "Inside" and L[2].type == "OnEdge": #this is only possible using the duo populate call, 3 points total
        inside_face = L[0].faces[0]
        redundant_faces.append(inside_face)
        assert( len(L) == 3 )
#         print("hey")
        if f[inside_face][0] not in L[2].between_vertices:
            assert( L[2].between_vertices != [])
            f2 = np.append(f2, np.array([[ f[inside_face][0] ,  f[inside_face][1] ,count_v ]]),axis = 0 )
            f2 = np.append(f2, np.array([[ f[inside_face][0] ,count_v ,  f[inside_face][2] ]]),axis = 0 )
            f2 = np.append(f2, np.array([[ count_v, count_v+3, f[inside_face][2] ]]),axis = 0 )
            f2 = np.append(f2, np.array([[ count_v+3, count_v+2, f[inside_face][2] ]]),axis = 0 )
            f2 = np.append(f2, np.array([[ count_v, f[inside_face][1], count_v+1 ]]),axis = 0 )
            f2 = np.append(f2, np.array([[ count_v+1, f[inside_face][1], count_v+2 ]]),axis = 0 )
        elif f[inside_face][1] not in L[2].between_vertices:
            assert( L[2].between_vertices != [])
            f2 = np.append(f2, np.array([[ f[inside_face][1] ,  f[inside_face][2] ,count_v ]]),axis = 0 )
            f2 = np.append(f2, np.array([[ f[inside_face][1] ,count_v ,  f[inside_face][0] ]]),axis = 0 )
            f2 = np.append(f2, np.array([[ count_v, count_v+3, f[inside_face][0] ]]),axis = 0 )
            f2 = np.append(f2, np.array([[ count_v+3, count_v+2, f[inside_face][0] ]]),axis = 0 )
            f2 = np.append(f2, np.array([[ count_v, f[inside_face][2], count_v+1 ]]),axis = 0 )
            f2 = np.append(f2, np.array([[ count_v+1, f[inside_face][2], count_v+2 ]]),axis = 0 )
        elif f[inside_face][2] not in L[2].between_vertices:
            assert( L[2].between_vertices != [])
            f2 = np.append(f2, np.array([[ f[inside_face][2] ,  f[inside_face][0] ,count_v ]]),axis = 0 )
            f2 = np.append(f2, np.array([[ f[inside_face][2] ,count_v ,  f[inside_face][1] ]]),axis = 0 )
            f2 = np.append(f2, np.array([[ count_v, count_v+3, f[inside_face][1] ]]),axis = 0 )
            f2 = np.append(f2, np.array([[ count_v+3, count_v+2, f[inside_face][1] ]]),axis = 0 )
            f2 = np.append(f2, np.array([[ count_v, f[inside_face][0], count_v+1 ]]),axis = 0 )
            f2 = np.append(f2, np.array([[ count_v+1, f[inside_face][0], count_v+2 ]]),axis = 0 )
            
        
    if L[0].type == "Inside" and L[1].type == "Inside" and L[2].type == "Inside":
        inside_face = L[0].faces[0]
        redundant_faces.append(inside_face)
        pos, neg, on_plane = [], [], []
        for i in range(3):
            t = where_is_i_vertex_of_face(i,inside_face)
            if t == 1: 
                pos.append(i)
            elif t == -1:
                neg.append(i)
            elif t == 0:
                on_plane.append(i)
            else:
                assert( False )

        assert(len(on_plane)<=1)
        if len(on_plane) == 1:
#             print("pos", pos)
#             print("neg", neg)
            assert(len(pos) == 1 and len(neg) == 1)
            if distance(L[0].position,v[f[inside_face][on_plane[0]]]) <= distance(L[2].position,v[f[inside_face][on_plane[0]]]):
                I_close = 0; I_far = 2;
            else:
                I_close = 2; I_far = 0;

            f2 = np.append(f2, np.array([[ f[inside_face][on_plane] , f[inside_face][neg] ,count_v+I_close ]]),axis = 0 )
            f2 = np.append(f2, np.array([[ f[inside_face][on_plane] , count_v+I_close ,  f[inside_face][pos]  ]]),axis = 0 )
            f2 = np.append(f2, np.array([[ count_v+I_far, f[inside_face][neg], f[inside_face][pos] ]]),axis = 0 )
            f2 = np.append(f2, np.array([[ count_v+I_close, f[inside_face][neg], count_v+3 ]]),axis = 0 )
            f2 = np.append(f2, np.array([[ count_v+I_far, count_v+3 , f[inside_face][neg] ]]),axis = 0 )
            f2 = np.append(f2, np.array([[ count_v+I_far, f[inside_face][pos], count_v+1 ]]),axis = 0 )
            f2 = np.append(f2, np.array([[ count_v+I_far, f[inside_face][pos], count_v+1 ]]),axis = 0 )
            f2 = np.append(f2, np.array([[ count_v+I_close, count_v+1, f[inside_face][pos]  ]]),axis = 0 )
        
    true_f = np.array([f2[i] for i in range(len(f2)) if i not in redundant_faces ])
    return L, v2, true_f, f2, redundant_faces


# Importing the model

In [ ]:

from Elements.definitions import MODEL_DIR

obj_to_import = MODEL_DIR / "Arm5.fbx"

# obj_to_import = MODEL_DIR /  "PlaneWith3Bones.fbx" # simple plane model with 3 bones
# obj_to_import = MODEL_DIR /  "3Cylinder4bones.fbx" # simple cylinder model with 4 bones

# scene = load(obj_to_import,processing=pyassimp.postprocess.aiProcessPreset_TargetRealtime_MaxQuality)
scene = pyassimp.load(str(obj_to_import)) # note that this takes some time!
mesh_id = 0



mesh = scene.meshes[mesh_id]
v = mesh.vertices
f = mesh.faces
b = mesh.bones
vw = vertex_weight(len(v))
vw.populate(b)




In [ ]:

from pathlib import Path
from Elements.definitions import PICKLES_DIR
import pickle

wanna_save = True
path_to_save = Path(PICKLES_DIR)/"arm5"

if wanna_save:
    with open(path_to_save/'vertices.pkl', 'wb') as file: pickle.dump(v, file)
    with open(path_to_save/'faces.pkl', 'wb') as file: pickle.dump(f, file)
    with open(path_to_save/'vw_id.pkl', 'wb') as file: pickle.dump(vw.id, file)
    with open(path_to_save/'vw_weight.pkl', 'wb') as file: pickle.dump(vw.weight, file)
    # with open(path_to_save/'MM0.pkl', 'wb') as file: pickle.dump(MM0, file)
    # with open(path_to_save/'BB.pkl', 'wb') as file: pickle.dump(BB, file)
    # with open(path_to_save/'newv.pkl', 'wb') as file: pickle.dump(newv, file)


In [ ]:
# IF pyassimp fails, then load the mesh from the pickles:

from pathlib import Path

wanna_load_pickles = True

path_to_load = PICKLES_DIR / "arm5"
if wanna_load_pickles:
    with open(path_to_load / 'vertices.pkl', 'rb') as file: v=pickle.load(file)
    with open(path_to_load / 'faces.pkl', 'rb') as file: f=pickle.load(file)
    vw = vertex_weight(len(v))
    with open(path_to_load / 'vw_id.pkl', 'rb') as file: vw.id = pickle.load(file)
    with open(path_to_load / 'vw_weight.pkl', 'rb') as file: vw.weight = pickle.load(file)
    
    


In [ ]:
start = time.time()
f = optimize_model(v,f)
v_n,e_n = face_neighbours(f)
end = time.time()
print("Time spend: ", end-start)

In [ ]:
# # WORKS NICE for plane model
# s1 = np.array([0.3,0.9,1]);      o1 = np.array([3.3,3.9,1]); 
# s2 = np.array([s1[0],s1[1],-1]);  o2 = np.array([o1[0],o1[1],-1]); 

# # WORKS NICE for cylinder model
# s1 = np.array([0.3,0.9,1]);      o1 = np.array([0.3,3.9,1]); 
# s2 = np.array([s1[0],s1[1],0]);  o2 = np.array([o1[0],o1[1],0]); 


# # WORKS NICE for arm5 model
s1 = np.array([15.5,37,-2.5]); o1 = np.array([-0.1,21.9,-10]);
s2 = np.array([0*s1[0],s1[1],s1[2]]);  o2 = np.array([o1[0],o1[1],0*o1[2]]); 


# # WORKS NICE for arm5 model
# s1 = np.array([15.5,37,-2.5]); o1 = np.array([-0.2,21.5,-10]);
# s2 = np.array([0*s1[0],s1[1],s1[2]]);  o2 = np.array([o1[0],o1[1],0*o1[2]]); 





# =================================================
# MULTIVECTORS FOR TRANSFORMATION
# =================================================
transform = True
transform = False

dd = [1 for i in range(len(b))]
tt = [1 for i in range(len(b))]
rr = [1 for i in range(len(b))]


rr[2] = axis_angle_to_rotor([0,0,1],0.9)
rr[1] = axis_angle_to_rotor([0,1,0],0.9)
rr[0] = axis_angle_to_rotor([1,0,0],1.7)
# tt[0] = vector_to_translator([0,2,0])
# dd[1] = scale_to_dilator(2)


# =================================================
# EVALUATING NEW VERTICES AFTER SKINNING
# =================================================


rotors = read_tree_ga(scene,mesh_id,tt,rr,dd,transform)
BB = [matrix_to_motor(b[i].offsetmatrix) for i in range(len(b))]
cgav = [up_vertex(vertex) for vertex in v]
newv = np.zeros([(len(v)),3])



for i in range(len(v)):
    for j in range(4):
        if vw.id[i][j] >=0:
            rotor =  (rotors[vw.id[i][j]])*BB[vw.id[i][j]] 
            temp = down_vertex(rotor*cgav[i]*~rotor)
            newv[i] = newv[i] + vw.weight[i][j]*temp
end = time.time()

# =================================================
# PLOTTING
# =================================================

# newv = newv/10

# print("TIME : ", end-start)
# p2 = mp.plot(newv2, true_f2,newv2[:, 1],shading={"scale": 2.5,"wireframe":True},return_plot=True)
p = mp.plot(newv, f,newv[:, 1],shading={"flat":False,"scale": 2.5,"wireframe":True},return_plot=True)
# p.save("2r.html")    



point_sizee = 2.5
p.add_points(np.array([s1]), shading={"point_size": point_sizee,"point_color": "red"} );
p.add_points(np.array([s2]), shading={"point_size": point_sizee,"point_color": "red"} );
p.add_lines(np.array([s1]), np.array([s2]), shading={"line_color": "red"});
p.add_points(np.array([o1]), shading={"point_size": point_sizee,"point_color": "green"} );
p.add_points(np.array([o2]), shading={"point_size": point_sizee,"point_color": "green"} );
p.add_lines(np.array([o1]), np.array([o2]), shading={"line_color": "green"});

In [ ]:
# # WORKS NICE for plane model
# s1 = np.array([0.3,0.9,1]);      o1 = np.array([3.3,3.9,1]); 
# s2 = np.array([s1[0],s1[1],-1]);  o2 = np.array([o1[0],o1[1],-1]); 

# # WORKS NICE for cylinder model
# s1 = np.array([0.3,0.9,1]);      o1 = np.array([0.3,3.9,1]); 
# s2 = np.array([s1[0],s1[1],0]);  o2 = np.array([o1[0],o1[1],0]); 

# # PROBLEM for arm5 model
# s1 = np.array([-0.1,21.9,10.5]); o1 = np.array([10.5,37,-2.5]);
# s2 = np.array([s1[0],s1[1],0]);  o2 = np.array([0,o1[1],o1[2]]); 

# # WORKS NICE for arm5 model
s1 = np.array([15.5,37,-2.5]); o1 = np.array([-0.1,21.9,-10]);
s2 = np.array([0*s1[0],s1[1],s1[2]]);  o2 = np.array([o1[0],o1[1],0*o1[2]]); 



# # WORKS NICE for arm5 model
# s1 = np.array([15.5,37,-2.5]); o1 = np.array([-0.2,21.5,-10]);
# s2 = np.array([0*s1[0],s1[1],s1[2]]);  o2 = np.array([o1[0],o1[1],0*o1[2]]); 





# =================================================
# MULTIVECTORS FOR TRANSFORMATION
# =================================================
transform = True
transform = False

dd = [1 for i in range(len(b))]
tt = [1 for i in range(len(b))]
rr = [1 for i in range(len(b))]


rr[2] = axis_angle_to_rotor([0,0,1],0.9)
rr[1] = axis_angle_to_rotor([0,1,0],0.9)
rr[0] = axis_angle_to_rotor([1,0,0],1.7)
# tt[0] = vector_to_translator([0,2,0])
# dd[1] = scale_to_dilator(2)


# =================================================
# EVALUATING NEW VERTICES AFTER SKINNING
# =================================================


rotors = read_tree_ga(scene,mesh_id,tt,rr,dd,transform)
BB = [matrix_to_motor(b[i].offsetmatrix) for i in range(len(b))]
cgav = [up_vertex(vertex) for vertex in v]
newv = np.zeros([(len(v)),3])



for i in range(len(v)):
    for j in range(4):
        if vw.id[i][j] >=0:
            rotor =  (rotors[vw.id[i][j]])*BB[vw.id[i][j]] 
            temp = down_vertex(rotor*cgav[i]*~rotor)
            newv[i] = newv[i] + vw.weight[i][j]*temp
end = time.time()

# =================================================
# PLOTTING
# =================================================

# newv = newv/10

# print("TIME : ", end-start)
# p2 = mp.plot(newv2, true_f2,newv2[:, 1],shading={"scale": 2.5,"wireframe":True},return_plot=True)
p = mp.plot(newv, f,newv[:, 1],shading={"flat":False,"scale": 2.5,"wireframe":True},return_plot=True)
# p.save("2r.html")    



point_sizee = 2.5
p.add_points(np.array([s1]), shading={"point_size": point_sizee,"point_color": "red"} );
p.add_points(np.array([s2]), shading={"point_size": point_sizee,"point_color": "red"} );
p.add_lines(np.array([s1]), np.array([s2]), shading={"line_color": "red"});
p.add_points(np.array([o1]), shading={"point_size": point_sizee,"point_color": "green"} );
p.add_points(np.array([o2]), shading={"point_size": point_sizee,"point_color": "green"} );
p.add_lines(np.array([o1]), np.array([o2]), shading={"line_color": "green"});


# draw = False
draw = True
faces_checked = []
faces_checked_result = []
# L, v2, true_f2, f2, redundant_faces = single_stich(o1,o2,s1,s2,newv,f,draw, 0.05)
L, v2, true_f2, f2, redundant_faces = single_stich(o1,o2,s1,s2,newv,f,draw, 0.3)
faces_checked = []
faces_checked_result = []
start = time.time()
L, v2, true_f2, f2, redundant_faces = single_stich(o1,o2,s1,s2,newv,f,draw, 0.3)
end = time.time()
print(end-start)


p2 = mp.plot(v2, true_f2,v2[:, 1],shading={"flat":False,"scale": 2.5,"wireframe":True,"side": "FrontSide"},return_plot=True);



In [ ]:
# # WORKS NICE for plane model
# s1 = np.array([0.3,0.9,1]);      o1 = np.array([3.3,3.9,1]); 
# s2 = np.array([s1[0],s1[1],-1]);  o2 = np.array([o1[0],o1[1],-1]); 

# # WORKS NICE for cylinder model
# s1 = np.array([0.3,0.9,1]);      o1 = np.array([0.3,3.9,1]); 
# s2 = np.array([s1[0],s1[1],0]);  o2 = np.array([o1[0],o1[1],0]); 

# # PROBLEM for arm5 model
# s1 = np.array([-0.1,21.9,10.5]); o1 = np.array([10.5,37,-2.5]);
# s2 = np.array([s1[0],s1[1],0]);  o2 = np.array([0,o1[1],o1[2]]); 

# # WORKS NICE for arm5 model
# s1 = np.array([15.5,37,-2.5]); o1 = np.array([-0.1,21.9,-10]);
# s2 = np.array([0*s1[0],s1[1],s1[2]]);  o2 = np.array([o1[0],o1[1],0*o1[2]]); 



# # WORKS NICE for arm5 model
s1 = np.array([15.5,37,-2.5]); o1 = np.array([-0.2,21.5,-10]);
s2 = np.array([0*s1[0],s1[1],s1[2]]);  o2 = np.array([o1[0],o1[1],0*o1[2]]); 





# =================================================
# MULTIVECTORS FOR TRANSFORMATION
# =================================================
transform = True
transform = False

dd = [1 for i in range(len(b))]
tt = [1 for i in range(len(b))]
rr = [1 for i in range(len(b))]


rr[2] = axis_angle_to_rotor([0,0,1],0.9)
rr[1] = axis_angle_to_rotor([0,1,0],0.9)
rr[0] = axis_angle_to_rotor([1,0,0],1.7)
# tt[0] = vector_to_translator([0,2,0])
# dd[1] = scale_to_dilator(2)


# =================================================
# EVALUATING NEW VERTICES AFTER SKINNING
# =================================================


rotors = read_tree_ga(scene,mesh_id,tt,rr,dd,transform)
BB = [matrix_to_motor(b[i].offsetmatrix) for i in range(len(b))]
cgav = [up_vertex(vertex) for vertex in v]
newv = np.zeros([(len(v)),3])



for i in range(len(v)):
    for j in range(4):
        if vw.id[i][j] >=0:
            rotor =  (rotors[vw.id[i][j]])*BB[vw.id[i][j]] 
            temp = down_vertex(rotor*cgav[i]*~rotor)
            newv[i] = newv[i] + vw.weight[i][j]*temp
# end = time.time()

# =================================================
# PLOTTING
# =================================================

# newv = newv/10

# print("TIME : ", end-start)
# p2 = mp.plot(newv2, true_f2,newv2[:, 1],shading={"scale": 2.5,"wireframe":True},return_plot=True)
# p = mp.plot(newv, f,newv[:, 1],shading={"flat":False,"scale": 2.5,"wireframe":True},return_plot=True)
# p.save("2r.html")    



# point_sizee = 2.5
# p.add_points(np.array([s1]), shading={"point_size": point_sizee,"point_color": "red"} );
# p.add_points(np.array([s2]), shading={"point_size": point_sizee,"point_color": "red"} );
# p.add_lines(np.array([s1]), np.array([s2]), shading={"line_color": "red"});
# p.add_points(np.array([o1]), shading={"point_size": point_sizee,"point_color": "green"} );
# p.add_points(np.array([o2]), shading={"point_size": point_sizee,"point_color": "green"} );
# p.add_lines(np.array([o1]), np.array([o2]), shading={"line_color": "green"});


draw = False
# draw = True
faces_checked = []
faces_checked_result = []
# L, v2, true_f2, f2, redundant_faces = single_stich(o1,o2,s1,s2,newv,f,draw, 0.05)
# L, v2, true_f2, f2, redundant_faces = single_stich(o1,o2,s1,s2,newv,f,draw, 0.3)
# faces_checked = []
# faces_checked_result = []
start = time.time()
L, v2, true_f2, f2, redundant_faces = single_stich(o1,o2,s1,s2,newv,f,draw, 0.3)
end = time.time()
print("total time:", end-start)


# p2 = mp.plot(v2, true_f2,v2[:, 1],shading={"flat":False,"scale": 2.5,"wireframe":True,"side": "FrontSide"},return_plot=True);




In [ ]:
def deform(v,a=0):
    transform = True
#     transform = False

    dd = [1 for i in range(len(b))]
    tt = [1 for i in range(len(b))]
    rr = [1 for i in range(len(b))]

    rr[0] = axis_angle_to_rotor([1,0,0],1.7)
    if a==0:
        ;
#         print("Original")
    elif a==2:
#         print("Rotation")
        rr[1] = axis_angle_to_rotor([0,1,0],1)
        rr[2] = axis_angle_to_rotor([0,1,0],-1)
    elif a==1:
#         print("Translation")
        tt[1] = vector_to_translator([-18,0,0])
    elif a==3:
#         print("Dilation")
        dd[1] = scale_to_dilator(1.5)
    elif a==4:
        rr[1] = axis_angle_to_rotor([0,1,0],1)
    


#     rr[1] = axis_angle_to_rotor([1,1,0],0.7)
#     tt[1] = vector_to_translator([-13,0,0])
#     dd[1] = scale_to_dilator(1.7)
    rotors = read_tree_ga(scene,mesh_id,tt,rr,dd,transform)


    cgav = [up_vertex(vertex) for vertex in v]
    newv = np.zeros([(len(v)),3])

#     start = time.time()
    for i in range(len(v)):
        for j in range(4):
            if vw.id[i][j] >=0:
                rotor =  (rotors[vw.id[i][j]])*BB[vw.id[i][j]] 
                temp = down_vertex(rotor*cgav[i]*~rotor)
                newv[i] = newv[i] + vw.weight[i][j]*temp
#     end = time.time()
    return newv
# =================================================
# PLOTTING
# =================================================

newv2 = deform(v2,0)

p2 = mp.plot(newv2, true_f2,newv2[:, 1],shading={"scale": 2.5,"wireframe":False,"side": "FrontSide"},return_plot=True)
# p2 = mp.plot(newv, f,newv[:, 1],shading={"scale": 2.5,"wireframe":True},return_plot=True)

# p2.save("voila.html")

In [ ]:
def tear_steps(a=1):
    point_sizee = 2.5
    if a==1:
        p = mp.plot(newv, f,newv[:, 1],shading={"flat":False,"scale": 2.5,"wireframe":True},return_plot=True)
        p.add_points(np.array([s1]), shading={"point_size": point_sizee,"point_color": "red"} );
        p.add_points(np.array([s2]), shading={"point_size": point_sizee,"point_color": "red"} );
        p.add_lines(np.array([s1]), np.array([s2]), shading={"line_color": "red"});
        p.add_points(np.array([o1]), shading={"point_size": point_sizee,"point_color": "green"} );
        p.add_points(np.array([o2]), shading={"point_size": point_sizee,"point_color": "green"} );
        p.add_lines(np.array([o1]), np.array([o2]), shading={"line_color": "green"});
    elif a == 2:
            p = mp.plot(finalv, finalf,finalv[:, 1],shading={"flat": False, "scale": 2.5,"wireframe":True},return_plot=True)
            p.add_lines(P3, P1, shading={"line_color": "red"});
            p.add_lines(P3, P2, shading={"line_color": "red"});
            p.add_lines(P1,P2, shading={"line_color": "red"});
            if len(finalL):
                p.add_points(np.array([p.position for p in finalL]),shading={"point_size": point_sizee,"point_color": "yellow"})
    elif a == 3: 
        p2 = mp.plot(v2, true_f2,v2[:, 1],shading={"flat":False,"scale": 2.5,"wireframe":True,"side": "FrontSide"},return_plot=True);

# tear_steps(2)
print("\n\n")
from ipywidgets import interact, interactive, fixed, interact_manual, widgets
interact(tear_steps, a=widgets.ToggleButtons(options=[('Scalpels', 1), ('Intersection Points', 2), ('Tear', 3)],value=1,description='State:') );
print("\n\n")

In [ ]:
def colorize(v,master_bone_id=0):
    c = np.zeros([len(v)])
    for i in range(len(v)):
        for j in range(4):
            if vw.id[i][j] == master_bone_id:
                c[i] = vw.weight[i][j]
                break
    return c

In [ ]:
def test(a=0):
    if a in [0,1,2,3]:
        newv2 = deform(newv,a)
        p2 = mp.plot(newv2,f,newv2[:, 1],shading={"flat":False,"scale": 2.5,"wireframe":True,"side": "FrontSide"},return_plot=True)
#     elif a == 4:
#         newv3 = deform(v2,a-4)
#         p2 = mp.plot(newv3, true_f2,newv3[:, 1],shading={"flat":False,"scale": 2.5,"wireframe":True,"colormap": "viridis","side": "FrontSide"},return_plot=True)
    elif a in [4,5,6,7]:
        c = -colorize(v2,1)
        newv3 = deform(v2,a-4)
#         p2 = mp.plot(newv3, true_f2,newv3[:, 1],shading={"flat":False,"scale": 2.5,"wireframe":True,"colormap": "viridis","side": "FrontSide"},return_plot=True)
        p2 = mp.plot(newv3, true_f2,c=c,shading={"flat":False,"scale": 2.5,"wireframe":True,"colormap": "autumn","side": "FrontSide"},return_plot=True)
    else:
        c = -colorize(v2,1)
        newv3 = deform(v2,a-8)
#         p2 = mp.plot(newv3, true_f2,newv3[:, 1],shading={"flat":False,"scale": 2.5,"wireframe":False,"colormap": "autumn","side": "FrontSide"},return_plot=True)
        p2 = mp.plot(newv3, true_f2,c,shading={"flat":False,"scale": 2.5,"wireframe":False,"colormap": "autumn","side": "FrontSide"},return_plot=True)




# 'Wistia', summer, autumn, viridis

drop = widgets.ToggleButtons(
    options=[('Original', 0), ('Translation', 1), ('Rotation', 2),('Dilation', 3),
            ('Tear ', 4), ('Tear + Translation', 5), ('Tear + Rotation', 6),('Tear + Dilation', 7),
            ('Tear (No Wireframe)', 8), ('Tear+Translation (NW)', 9), ('Tear+Rotation (NW)', 10),('Tear+Dilation (NW)',11)],
    value=0,
    description='State:',layout=widgets.Layout(width='78%')
)



# RESULTS

In [ ]:
print("\n\n")
zzz = interact(test, a=drop);
print("\n\n")